In [1]:
import pandas as pd
from datetime import datetime, date, timedelta
from collections import namedtuple
from typing import Optional
import numpy as np
from typing import Dict, Generator, Tuple, Sequence, Optional

In [2]:
Disposition = namedtuple("Disposition", ("rate", "days"))

class Parameters:
    """Parameters."""
# 'covid_census_value', 'covid_census_date', 'total_covid_beds', 'icu_covid_beds', 'covid_ventilators', 'non_icu', 'icu', 'relative_contact_rate', and 'ventilators'
    def __init__(
        self,
        *,
        covid_census_value: int = 50, # used to be current_hospitalized
        covid_census_date: datetime.date, # added by Health Catalyst team
        beds_borrow: bool = True,
        total_covid_beds: int,
        icu_covid_beds: int,
        covid_ventilators: int,
        non_icu: Disposition,
        icu: Disposition,
        relative_contact_rate: float,
        ventilators: Disposition, # used to be ventilated
        current_date: datetime.date = date.today() - timedelta(hours=6),
        social_distancing_is_implemented: bool = False,
        mitigation_date: datetime.date = date.today()  - timedelta(hours=6),
        date_first_hospitalized: datetime.date = None,
        first_hospitalized_date_known: bool = False,
        doubling_time: Optional[float] = None,
        infectious_days: int = 14,
        market_share: float = 1.0,
        max_y_axis: Optional[int] = None,
        max_y_axis_set: bool = False,
        n_days: int = 100,
        population: int = 3600000,
        recovered: int = 0,
        # Added by the Health Catalyst Team
        author: str = "Jane Doe",
        scenario: str = "Scenario Name",
        # PPE Params
        masks_n95: int = 5,
        masks_surgical: int = 7,
        face_shield: int = 5,
        gloves: int = 10,
        gowns: int = 10,
        other_ppe: int = 2,
        masks_n95_icu: int = 5,
        masks_surgical_icu: int = 7,
        face_shield_icu: int = 5,
        gloves_icu: int = 10,
        gowns_icu: int = 10,
        other_ppe_icu: int = 2,
        
        # Staffing Params
        # Non-ICU
        nurses: int = 6,
        physicians: int = 20,
        advanced_practice_providers: int= 20,
        healthcare_assistants: int = 10,
        other_staff=10,
        # ICU
        nurses_icu: int = 2,
        physicians_icu: int = 12,
        advanced_practice_providers_icu: int = 12,
        healthcare_assistants_icu: int = 10,
        other_staff_icu=10,
        # Shift Duration
        shift_duration: int = 12,
    ):
        self.covid_census_value = 50
        self.covid_census_date = date(2020, 4, 1)
        self.current_date = date.today()
        self.relative_contact_rate = 0.45

        self.beds_borrow = True
        self.total_covid_beds = 300
        self.icu_covid_beds = 30
        self.covid_ventilators = 10

        self.non_icu = Disposition(.025, 7)
        self.icu = Disposition(.0075, 9)

        self.ventilators = Disposition(.0005, 9)

        self.population = 3600000

        self.mitigation_date = date(2020, 3, 15)
        self.social_distancing_is_implemented = social_distancing_is_implemented

        self.date_first_hospitalized = date(2020, 3, 5)
        self.first_hospitalized_date_known = True
        
        self.doubling_time = 4

        self.infectious_days = 14
        self.market_share = 0.15 
        self.n_days = 100 
        self.recovered = 0        

        self.dispositions = {
            "total": non_icu,
            "non_icu": non_icu,
            "icu": icu,
            "ventilators": ventilators,
        }
        
        # PPE Params
        self.masks_n95 = masks_n95
        self.masks_surgical = masks_surgical
        self.face_shield = face_shield
        self.gloves = gloves 
        self.gowns = gowns
        self.other_ppe = other_ppe
        self.masks_n95_icu = masks_n95_icu
        self.masks_surgical_icu = masks_surgical_icu
        self.face_shield_icu = face_shield_icu
        self.gloves_icu = gloves_icu
        self.gowns_icu = gowns_icu
        self.other_ppe_icu = other_ppe_icu
        
        # Staffing Params
        # Non-ICU
        self.nurses = nurses
        self.physicians = physicians
        self.advanced_practice_providers = advanced_practice_providers
        self.healthcare_assistants = healthcare_assistants
        self.other_staff = other_staff
        # ICU
        self.nurses_icu = nurses_icu
        self.physicians_icu = physicians_icu
        self.advanced_practice_providers_icu = advanced_practice_providers_icu
        self.healthcare_assistants_icu = healthcare_assistants_icu
        self.other_staff_icu = other_staff_icu
        # Shift Duration
        self.shift_duration = shift_duration

        self.actuals_labels = {
            "total_admissions_actual": "Total ",
            "non_icu_admissions_actual": "Non-ICU ",
            "icu_admissions_actual": "ICU ",
            "intubated_actual": "Intubated ",
            "total_census_actual": "Total ",
            "non_icu_census_actual": "Non-ICU ",
            "icu_census_actual": "ICU ",
            "ventilators_in_use_actual": "Ventialtors In Use ",
        }
        
        self.admits_patient_chart_desc = {
            "Non-ICU": "Non-ICU COVID-19 census peak at",
            "ICU": "ICU COVID-19 Admissions peak at",
            "Ventilators": "COVID-19 Ventilators peak at",
            "Total": "Total COVID-19 Admissions peaks at",
        }

        self.census_patient_chart_desc = {
            "Non-ICU": "Non-ICU COVID-19 Census peaks at",
            "ICU": "ICU COVID-19 Census peaks at",
            "Ventilators": "COVID-19 Ventilator usage peaks at",
            "Total": "Total COVID-19 Census peaks at"
        }
        
        self.beds_chart_desc = {
            "Non-ICU": "Non-ICU COVID-19 Beds",
            "ICU": "ICU COVID-19 Beds",
            "Ventilators": "COVID-19 Ventilators",
            "Total": "Total COVID-19 Beds",
        }

        self.ppe_labels = {
            "total": "Total",
            "non_icu": "Non-ICU",
            "icu": "ICU",
            "masks_n95": {
                "label": "Masks - N95",
                "col1_name": "masks_n95_total",
                "col2_name": "masks_n95_non_icu",
                "col3_name": "masks_n95_icu",
            },
            "masks_surgical": {
                "label": "Masks - Surgical",
                "col1_name": "masks_surgical_total",
                "col2_name": "masks_surgical_non_icu",
                "col3_name": "masks_surgical_icu",
            },
            "face_shield": {
                "label": "Face Shields",
                "col1_name": "face_shield_total",
                "col2_name": "face_shield_non_icu",
                "col3_name": "face_shield_icu",
            },
            "gloves": {
                "label": "Gloves",
                "col1_name": "gloves_total",
                "col2_name": "gloves_non_icu",
                "col3_name": "gloves_icu",
            },
            "gowns": {
                "label": "Gowns",
                "col1_name": "gowns_total",
                "col2_name": "gowns_non_icu",
                "col3_name": "gowns_icu",
            },
            "other_ppe": {
                "label": "Other PPE",
                "col1_name": "other_ppe_total",
                "col2_name": "other_ppe_non_icu",
                "col3_name": "other_ppe_icu",
            },
        }

        self.staffing_labels = {
            "total": "Total",
            "icu": "ICU",
            "non_icu": "Non-ICU",
            "nurses": {
                "label": "Nurses",
                "col1_name": "nurses_total",
                "col2_name": "nurses_non_icu",
                "col3_name": "nurses_icu",
            },
            "physicians": {
                "label": "Physicians",
                "col1_name": "physicians_total",
                "col2_name": "physicians_non_icu",
                "col3_name": "physicians_icu",
            },
            "advanced_practice_providers": {
                "label": "Advanced Practice Providers",
                "col1_name": "advanced_practice_providers_total",
                "col2_name": "advanced_practice_providers_non_icu",
                "col3_name": "advanced_practice_providers_icu",
            },
            "healthcare_assistants": {
                "label": "Healthcare Assistants",
                "col1_name": "healthcare_assistants_total",
                "col2_name": "healthcare_assistants_non_icu",
                "col3_name": "healthcare_assistants_icu",
            },
            "other_staff": {
                "label": "Other Staff",
                "col1_name": "other_staff_total",
                "col2_name": "other_staff_non_icu",
                "col3_name": "other_staff_icu",
            },
        }
        
def get_defaults():
    return Parameters(
        population=3600000,
        covid_census_value=10,
        covid_census_date=(datetime.utcnow() - timedelta(hours=6)).date(),
        total_covid_beds=300,
        icu_covid_beds=30,
        covid_ventilators=10,
        date_first_hospitalized=date(2020,3,1),
        doubling_time=4.0,
        non_icu=Disposition(0.025, 7),
        icu=Disposition(0.0075, 9),
        infectious_days=14,
        market_share=0.15,
        n_days=100,
        relative_contact_rate=0.3,
        ventilators=Disposition(0.005, 10),
        masks_n95=5,
        masks_surgical=7,
        face_shield=5,
        gloves=10,
        gowns=10,
        other_ppe=2,
        masks_n95_icu=5,
        masks_surgical_icu=7,
        face_shield_icu=5,
        gloves_icu=10,
        gowns_icu=10,
        other_ppe_icu=2,
        # Staffing Params
        # Non-ICU
        nurses = 6,
        physicians = 20,
        advanced_practice_providers = 20,
        healthcare_assistants = 10,
        other_staff=10,
        # ICU
        nurses_icu = 2,
        physicians_icu= 12,
        advanced_practice_providers_icu = 12,
        healthcare_assistants_icu = 10,
        other_staff_icu=10,
        # Shift Duration
        shift_duration = 12,
    )

In [3]:
p = get_defaults()
vars(p)

{'covid_census_value': 50,
 'covid_census_date': datetime.date(2020, 4, 1),
 'current_date': datetime.date(2020, 4, 15),
 'relative_contact_rate': 0.45,
 'beds_borrow': True,
 'total_covid_beds': 300,
 'icu_covid_beds': 30,
 'covid_ventilators': 10,
 'non_icu': Disposition(rate=0.025, days=7),
 'icu': Disposition(rate=0.0075, days=9),
 'ventilators': Disposition(rate=0.0005, days=9),
 'population': 3600000,
 'mitigation_date': datetime.date(2020, 3, 15),
 'social_distancing_is_implemented': False,
 'date_first_hospitalized': datetime.date(2020, 3, 5),
 'first_hospitalized_date_known': True,
 'doubling_time': 4,
 'infectious_days': 14,
 'market_share': 0.15,
 'n_days': 100,
 'recovered': 0,
 'dispositions': {'total': Disposition(rate=0.025, days=7),
  'non_icu': Disposition(rate=0.025, days=7),
  'icu': Disposition(rate=0.0075, days=9),
  'ventilators': Disposition(rate=0.005, days=10)},
 'masks_n95': 5,
 'masks_surgical': 7,
 'face_shield': 5,
 'gloves': 10,
 'gowns': 10,
 'other_ppe':

In [6]:
# Model file at HC 1.7.1
CHANGE_DATE = date(year=2020, month=4, day=1)
DATE_FORMAT = "%b, %d"  # see https://strftime.org
EPSILON = 1.0e-7

class SimSirModel:

    def __init__(self, p: Parameters):

        self.rates = {
            key: d.rate
            for key, d in p.dispositions.items()
        }

        self.days = {
            key: d.days
            for key, d in p.dispositions.items()
        }

        self.keys = ("susceptible", "infected", "recovered")

        # An estimate of the number of infected people on the day that
        # the first hospitalized case is seen
        #
        # Note: this should not be an integer.
        infected = (
            1.0 / p.market_share / p.non_icu.rate
        )

        susceptible = p.population - infected

        gamma = 1.0 / p.infectious_days
        self.gamma = gamma

        self.susceptible = susceptible
        self.infected = infected
        self.recovered = p.recovered

        if p.doubling_time is not None:
            # Back-projecting to when the first hospitalized case would have been admitted

            intrinsic_growth_rate = get_growth_rate(p.doubling_time)
            self.beta = get_beta(intrinsic_growth_rate,  gamma, self.susceptible, 0.0)
            self.beta_t = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, p.relative_contact_rate)

            if p.mitigation_date is None:
                self.i_day = 0 # seed to the full length
                temp_n_days = p.n_days
                p.n_days = 1000
                raw = self.run_projection(p, [(self.beta, p.n_days)])
                self.i_day = i_day = int(get_argmin_ds(raw["census_non_icu"], p.covid_census_value))
                p.n_days = temp_n_days

                self.raw = self.run_projection(p, self.gen_policy(p))

            else:
                projections = {}
                best_i_day = -1
                best_i_day_loss = float('inf')
                temp_n_days = p.n_days
                p.n_days = 1000
                for i_day in range(90):
                    self.i_day = i_day
                    raw = self.run_projection(p, self.gen_policy(p))


                    # Don't fit against results that put the peak before the present day
                    if raw["census_non_icu"].argmax() < i_day:
                        continue

                    loss = get_loss(raw["census_non_icu"][i_day], p.covid_census_value)
                    if loss < best_i_day_loss:
                        best_i_day_loss = loss
                        best_i_day = i_day
                p.n_days = temp_n_days
                self.i_day = best_i_day
                raw = self.run_projection(p, self.gen_policy(p))
                self.raw = raw

        elif p.date_first_hospitalized is not None:
            # Fitting spread parameter to observed hospital census (dates of 1 patient and today)
            self.i_day = (p.covid_census_date - p.date_first_hospitalized).days
            self.covid_census_value = p.covid_census_value

            # Make an initial coarse estimate
            dts = np.linspace(1, 15, 15)
            min_loss = self.get_argmin_doubling_time(p, dts)

            # Refine the coarse estimate
            for iteration in range(4):
                dts = np.linspace(dts[min_loss-1], dts[min_loss+1], 15)
                min_loss = self.get_argmin_doubling_time(p, dts)

            p.doubling_time = dts[min_loss]

            intrinsic_growth_rate = get_growth_rate(p.doubling_time)
            self.beta = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, 0.0)
            self.beta_t = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, p.relative_contact_rate)
            self.raw = self.run_projection(p, self.gen_policy(p))

            self.population = p.population
        else:
            raise AssertionError('doubling_time or date_first_hospitalized must be provided.')

        self.raw["date"] = self.raw["day"].astype("timedelta64[D]") + np.datetime64(p.covid_census_date)

        self.raw_df = pd.DataFrame(data=self.raw)
        self.dispositions_df = pd.DataFrame(data={
            'day': self.raw['day'],
            'date': self.raw['date'],
            'ever_non_icu': self.raw['ever_non_icu'],
            'ever_icu': self.raw['ever_icu'],
            'ever_ventilators': self.raw['ever_ventilators'],
        })
        self.admits_df = pd.DataFrame(data={
            'day': self.raw['day'],
            'date': self.raw['date'],
            'non_icu': self.raw['admits_non_icu'],
            'icu': self.raw['admits_icu'],
            'ventilators': self.raw['admits_ventilators'],
            'total': self.raw['admits_total']
        })
        self.census_df = pd.DataFrame(data={
            'day': self.raw['day'],
            'date': self.raw['date'],
            'non_icu': self.raw['census_non_icu'],
            'icu': self.raw['census_icu'],
            'ventilators': self.raw['census_ventilators'],
            'total': self.raw['census_total'],
        })
        self.beds_df = build_beds_df(self.census_df, p)
        self.ppe_df = build_ppe_df(self.census_df, p)
        self.staffing_df = build_staffing_df(self.census_df, p)

        self.infected = self.raw_df['infected'].values[self.i_day]
        self.susceptible = self.raw_df['susceptible'].values[self.i_day]
        self.recovered = self.raw_df['recovered'].values[self.i_day]

        self.intrinsic_growth_rate = intrinsic_growth_rate

        # r_t is r_0 after distancing
        self.r_t = self.beta_t / gamma * susceptible
        self.r_naught = self.beta / gamma * susceptible

        doubling_time_t = 1.0 / np.log2(
            self.beta_t * susceptible - gamma + 1)
        self.doubling_time_t = doubling_time_t

        self.sim_sir_w_date_df = build_sim_sir_w_date_df(self.raw_df, p.covid_census_date, self.keys)

        self.sim_sir_w_date_floor_df = build_floor_df(self.sim_sir_w_date_df, self.keys)
        self.admits_floor_df = build_floor_df(self.admits_df, p.dispositions.keys())
        self.census_floor_df = build_floor_df(self.census_df, p.dispositions.keys())
        self.beds_floor_df = build_floor_df(self.beds_df, p.dispositions.keys())
        self.ppe_floor_df = build_floor_df(self.ppe_df, self.ppe_df.columns[2:])
        self.staffing_floor_df = build_floor_df(self.staffing_df, self.staffing_df.columns[2:])

        self.daily_growth_rate = get_growth_rate(p.doubling_time)
        self.daily_growth_rate_t = get_growth_rate(self.doubling_time_t)

    def get_argmin_doubling_time(self, p: Parameters, dts):
        losses = np.full(dts.shape[0], np.inf)
        for i, i_dt in enumerate(dts):
            intrinsic_growth_rate = get_growth_rate(i_dt)
            self.beta = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, 0.0)
            self.beta_t = get_beta(intrinsic_growth_rate, self.gamma, self.susceptible, p.relative_contact_rate)

            raw = self.run_projection(p, self.gen_policy(p))

            # Skip values the would put the fit past peak
            peak_admits_day = raw["admits_non_icu"].argmax()
            if peak_admits_day < 0:
                continue

            predicted = raw["census_non_icu"][self.i_day]
            loss = get_loss(self.covid_census_value, predicted)
            losses[i] = loss

        min_loss = pd.Series(losses).argmin()
        return min_loss

    def gen_policy(self, p: Parameters) -> Sequence[Tuple[float, int]]:
        if p.mitigation_date is not None:
            mitigation_day = -(p.covid_census_date - p.mitigation_date).days
        else:
            mitigation_day = 0

        total_days = self.i_day + p.n_days

        if mitigation_day < -self.i_day:
            mitigation_day = -self.i_day

        pre_mitigation_days = self.i_day + mitigation_day
        post_mitigation_days = total_days - pre_mitigation_days

        return [
            (self.beta,   pre_mitigation_days),
            (self.beta_t, post_mitigation_days),
        ]

    def run_projection(self, p: Parameters, policy: Sequence[Tuple[float, int]]):
        raw = sim_sir(
            self.susceptible,
            self.infected,
            p.recovered,
            self.gamma,
            -self.i_day,
            policy
        )

        calculate_dispositions(raw, self.rates, p.market_share)
        calculate_admits(raw, self.rates)
        calculate_census(raw, self.days)

        return raw


def get_loss(current_hospitalized, predicted) -> float:
    """Squared error: predicted vs. actual current hospitalized."""
    return (current_hospitalized - predicted) ** 2.0


def get_argmin_ds(census, current_hospitalized: float) -> float:
    # By design, this forbids choosing a day after the peak
    # If that's a problem, see #381
    peak_day = census.argmax()
    losses = (census[:peak_day] - current_hospitalized) ** 2.0
    return losses.argmin()


def get_beta(
    intrinsic_growth_rate: float,
    gamma: float,
    susceptible: float,
    relative_contact_rate: float
) -> float:
    return (
        (intrinsic_growth_rate + gamma)
        / susceptible
        * (1.0 - relative_contact_rate)
    )


def get_growth_rate(doubling_time: Optional[float]) -> float:
    """Calculates average daily growth rate from doubling time."""
    if doubling_time is None or doubling_time == 0.0:
        return 0.0
    return (2.0 ** (1.0 / doubling_time) - 1.0)


def sir(
    s: float, i: float, r: float, beta: float, gamma: float, n: float
) -> Tuple[float, float, float]:
    """The SIR model, one time step."""
    s_n = (-beta * s * i) + s
    i_n = (beta * s * i - gamma * i) + i
    r_n = gamma * i + r
    scale = n / (s_n + i_n + r_n)
    return s_n * scale, i_n * scale, r_n * scale


def sim_sir(
    s: float, i: float, r: float, gamma: float, i_day: int, policies: Sequence[Tuple[float, int]]
):
    """Simulate SIR model forward in time, returning a dictionary of daily arrays
    Parameter order has changed to allow multiple (beta, n_days)
    to reflect multiple changing social distancing policies.
    """
    s, i, r = (float(v) for v in (s, i, r))
    n = s + i + r
    d = i_day

    total_days = 1
    for beta, days in policies:
        total_days += days

    d_a = np.empty(total_days, "int")
    s_a = np.empty(total_days, "float")
    i_a = np.empty(total_days, "float")
    r_a = np.empty(total_days, "float")

    index = 0
    for beta, n_days in policies:
        for _ in range(n_days):
            d_a[index] = d
            s_a[index] = s
            i_a[index] = i
            r_a[index] = r
            index += 1

            s, i, r = sir(s, i, r, beta, gamma, n)
            d += 1

    d_a[index] = d
    s_a[index] = s
    i_a[index] = i
    r_a[index] = r
    return {
        "day": d_a,
        "susceptible": s_a,
        "infected": i_a,
        "recovered": r_a,
        "ever_infected": i_a + r_a
    }


def build_sim_sir_w_date_df(
    raw_df: pd.DataFrame,
    current_date: datetime,
    keys: Sequence[str],
) -> pd.DataFrame:
    day = raw_df.day
    return pd.DataFrame({
        "day": day,
        "date": day.astype('timedelta64[D]') + np.datetime64(current_date),
        **{
            key: raw_df[key]
            for key in keys
        }
    })


def build_floor_df(df, keys):
    """Build floor sim sir w date."""
    return pd.DataFrame({
        "day": df.day,
        "date": df.date,
        **{
            key: np.floor(df[key])
            for key in keys
        }
    })


def calculate_dispositions(
    raw: Dict,
    rates: Dict[str, float],
    market_share: float,
):
    """Build dispositions dataframe of patients adjusted by rate and market_share."""
    for key, rate in rates.items():
        raw["ever_" + key] = raw["ever_infected"] * rate * market_share
        raw[key] = raw["ever_infected"] * rate * market_share


def calculate_admits(raw: Dict, rates):
    """Build admits dataframe from dispositions."""
    for key in rates.keys():
        ever = raw["ever_" + key]
        admit = np.empty_like(ever)
        admit[0] = np.nan
        admit[1:] = ever[1:] - ever[:-1]
        raw["admits_"+key] = admit
        raw[key] = admit
    raw['admits_total'] = np.floor(raw['admits_non_icu']) + np.floor(raw['admits_icu'])


def calculate_census(
    raw: Dict,
    lengths_of_stay: Dict[str, int],
):
    """Average Length of Stay for each disposition of COVID-19 case (total guesses)"""
    n_days = raw["day"].shape[0]
    for key, los in lengths_of_stay.items():
        cumsum = np.empty(n_days + los)
        cumsum[:los+1] = 0.0
        cumsum[los+1:] = raw["admits_" + key][1:].cumsum()

        census = cumsum[los:] - cumsum[:-los]
        raw["census_" + key] = census
    raw['census_total'] = np.floor(raw['census_non_icu']) + np.floor(raw['census_icu'])

def build_beds_df(
    census_df: pd.DataFrame,
    p,
) -> pd.DataFrame:
    """ALOS for each category of COVID-19 case (total guesses)"""
    beds_df = pd.DataFrame()
    beds_df["day"] = census_df["day"]
    beds_df["date"] = census_df["date"]

    # If hospitalized < 0 and there's space in icu, start borrowing if possible
    # If ICU < 0, raise alarms. No changes.
    beds_df["non_icu"] = p.total_covid_beds - p.icu_covid_beds - census_df["non_icu"]
    beds_df["icu"] = p.icu_covid_beds - census_df["icu"]
    beds_df["ventilators"] = p.covid_ventilators - census_df["ventilators"]
    beds_df["total"] = p.total_covid_beds - census_df["non_icu"] - census_df["icu"]
    # beds_df = beds_df.head(n_days)

    # Shift people to ICU if main hospital is full and ICU is not.
    # And vice versa
    if p.beds_borrow:
        new_hosp = []
        new_icu = []
        for row in beds_df.itertuples():
            if row.non_icu < 0 and row.icu > 0: # ICU to Non-ICU
                needed = min(abs(row.non_icu), row.icu)
                new_hosp.append(row.non_icu + needed)
                new_icu.append(row.icu - needed)
            elif row.non_icu > 0 and row.icu < 0: # Non-ICU to ICU
                needed = min(abs(row.icu), row.non_icu)
                new_hosp.append(row.non_icu - needed)
                new_icu.append(row.icu + needed)
            else: 
                new_hosp.append(row.non_icu)
                new_icu.append(row.icu)
        beds_df["non_icu"] = new_hosp
        beds_df["icu"] = new_icu
    return beds_df

def build_ppe_df(
    census_df: pd.DataFrame,
    p,
) -> pd.DataFrame:
    """ALOS for each category of COVID-19 case (total guesses)"""
    ppe_df = pd.DataFrame()
    ppe_df["day"] = census_df["day"]
    ppe_df["date"] = census_df["date"]

    fnic = np.floor(census_df.non_icu) # floored non-icu census
    ppe_df["masks_n95_non_icu"] = p.masks_n95 * fnic
    ppe_df["masks_surgical_non_icu"] = p.masks_surgical * fnic
    ppe_df["face_shield_non_icu"] = p.face_shield * fnic
    ppe_df["gloves_non_icu"] = p.gloves * fnic
    ppe_df["gowns_non_icu"] = p.gowns * fnic
    ppe_df["other_ppe_non_icu"] = p.other_ppe * fnic

    fic = np.floor(census_df.icu) # floored icu census
    ppe_df["masks_n95_icu"] = p.masks_n95_icu * fic
    ppe_df["masks_surgical_icu"] = p.masks_surgical_icu * fic
    ppe_df["face_shield_icu"] = p.face_shield_icu * fic
    ppe_df["gloves_icu"] = p.gloves_icu * fic
    ppe_df["gowns_icu"] = p.gowns_icu * fic
    ppe_df["other_ppe_icu"] = p.other_ppe_icu * fic
    
    ppe_df["masks_n95_total"] = ppe_df["masks_n95_non_icu"] + ppe_df["masks_n95_icu"]
    ppe_df["masks_surgical_total"] = ppe_df["masks_surgical_non_icu"] + ppe_df["masks_surgical_icu"]
    ppe_df["face_shield_total"] = ppe_df["face_shield_non_icu"] + ppe_df["face_shield_icu"]
    ppe_df["gloves_total"] = ppe_df["gloves_non_icu"] + ppe_df["gloves_icu"]
    ppe_df["gowns_total"] = ppe_df["gowns_non_icu"] + ppe_df["gowns_icu"]
    ppe_df["other_ppe_total"] = ppe_df["other_ppe_non_icu"] + ppe_df["other_ppe_icu"]

    return ppe_df


def build_staffing_df(
    census_df: pd.DataFrame,
    p,
) -> pd.DataFrame:
    """ALOS for each category of COVID-19 case (total guesses)"""
    staffing_df = pd.DataFrame()
    staffing_df["day"] = census_df["day"]
    staffing_df["date"] = census_df["date"]

    stf_mul = 24.0 / p.shift_duration # Staffing Multiplier
    fnic = np.floor(census_df.non_icu) # floored non-icu census
    fic = np.floor(census_df.icu) # floored icu census

    staffing_df["nurses_non_icu"] = np.ceil(np.ceil(
        fnic / p.nurses) * stf_mul) if p.nurses !=0 else 0
    staffing_df["physicians_non_icu"] = np.ceil(np.ceil(
        fnic / p.physicians) * stf_mul) if p.physicians != 0 else 0
    staffing_df["advanced_practice_providers_non_icu"] = np.ceil(np.ceil(
        fnic / p.advanced_practice_providers) * stf_mul) if p.advanced_practice_providers != 0 else 0
    staffing_df["healthcare_assistants_non_icu"] = np.ceil(np.ceil(
        fnic / p.healthcare_assistants) * stf_mul) if p.healthcare_assistants != 0 else 0
    staffing_df["other_staff_non_icu"] = np.ceil(np.ceil(
    fnic / p.other_staff) * stf_mul) if p.other_staff != 0 else 0

    staffing_df["nurses_icu"] = np.ceil(np.ceil(
        fic / p.nurses_icu) * stf_mul) if p.nurses_icu !=0 else 0
    staffing_df["physicians_icu"] = np.ceil(np.ceil(
        fic / p.physicians_icu) * stf_mul) if p.physicians_icu !=0 else 0
    staffing_df["advanced_practice_providers_icu"] = np.ceil(np.ceil(
        fic / p.advanced_practice_providers_icu) * stf_mul) if p.advanced_practice_providers_icu !=0 else 0
    staffing_df["healthcare_assistants_icu"] = np.ceil(np.ceil(
        fic / p.healthcare_assistants_icu) * stf_mul) if p.healthcare_assistants_icu !=0 else 0
    staffing_df["other_staff_icu"] = np.ceil(np.ceil(
        fic / p.other_staff_icu) * stf_mul) if p.other_staff_icu != 0 else 0

    staffing_df["nurses_total"] = np.ceil(staffing_df["nurses_non_icu"] + staffing_df["nurses_icu"])
    staffing_df["physicians_total"] = np.ceil(staffing_df["physicians_non_icu"] + staffing_df["physicians_icu"])
    staffing_df["advanced_practice_providers_total"] = np.ceil(staffing_df["advanced_practice_providers_non_icu"] + staffing_df["advanced_practice_providers_icu"])
    staffing_df["healthcare_assistants_total"] = np.ceil(staffing_df["healthcare_assistants_non_icu"] + staffing_df["healthcare_assistants_icu"])
    staffing_df["other_staff_total"] = np.ceil(staffing_df["other_staff_non_icu"] + staffing_df["other_staff_icu"])
    return staffing_df


In [7]:
m = SimSirModel(p)

In [8]:
m.admits_floor_df

,day,date,total,non_icu,icu,ventilators
0,-34,2020-02-27,NaN,NaN,NaN,NaN
1,-33,2020-02-28,0.0,0.0,0.0,0.0
2,-32,2020-02-29,0.0,0.0,0.0,0.0
3,-31,2020-03-01,0.0,0.0,0.0,0.0
4,-30,2020-03-02,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
130,96,2020-07-06,95.0,73.0,22.0,14.0
131,97,2020-07-07,91.0,70.0,21.0,14.0
132,98,2020-07-08,87.0,67.0,20.0,13.0
133,99,2020-07-09,83.0,64.0,19.0,12.0


In [9]:
doubling_time = 4
growth_rate = 2.0 ** (1.0 / doubling_time) - 1.0
growth_rate * 100

18.920711500272102

In [29]:
x0 = np.array([35240, 45071, 55674, 69660, 86570, 105363, 125415])
x1 = np.array([45071, 55674, 69660, 86570, 105363, 125415, 143444])
growth_rate = (x1 - x0) / x0
doubling_time = 1.0 / np.log2(growth_rate + 1)
gamma = 1/14
beta = growth_rate + gamma

for val in beta:
    print(val)

0.3504013296578563
0.3066796197745145
0.32264098655951223
0.31417907386899635
0.28851301176587074
0.26174205908552883
0.21518330571075459


In [12]:
35000*1.28


44800.0

In [28]:
# Make changes to model file
active_cases = np.array([34100, 43374, 53468, 66488, 82439, 100022, 118522])
cum_rec = np.array([668, 1096, 1400, 2107, 2807, 3612, 4685])
cum_dead = np.array([472, 601, 806, 1065, 1324, 1729, 2208])

# Doesn't change
population = 328200000
# Varies with time
for i0, r_act, cd in zip(x0, cum_rec, cum_dead):
    r_calc = i0 * 1/14
    s_calc = population - i0 - r_calc
    print("s: ", s_calc)
    print("i: ", i0)
    print("rc: ", r_calc)
    print("r_act: ", r_act+cd)
    print("  ")

s:  328162242.85714287
i:  35240
rc:  2517.1428571428573
r_act:  1140
  
s:  328151709.64285713
i:  45071
rc:  3219.3571428571427
r_act:  1697
  
s:  328140349.28571427
i:  55674
rc:  3976.714285714286
r_act:  2206
  
s:  328125364.28571427
i:  69660
rc:  4975.714285714285
r_act:  3172
  
s:  328107246.4285714
i:  86570
rc:  6183.571428571428
r_act:  4131
  
s:  328087111.0714286
i:  105363
rc:  7525.928571428572
r_act:  5341
  
s:  328065626.78571427
i:  125415
rc:  8958.214285714286
r_act:  6893
  


In [49]:
def sir(
    s: float, i: float, r: float, beta: float, gamma: float, n: float
) -> Tuple[float, float, float]:
    """The SIR model, one time step."""
    s_n = (-beta * s * i) + s
    i_n = (beta * s * i - gamma * i) + i
    r_n = gamma * i + r
    scale = n / (s_n + i_n + r_n)
    return s_n * scale, i_n * scale, r_n * scale


def sim_sir(
    s: float, i: float, r: float, gamma: float, i_day: int, beta: np.array
):
    """Simulate SIR model forward in time, returning a dictionary of daily arrays
    Parameter order has changed to allow multiple (beta, n_days)
    to reflect multiple changing social distancing policies.
    """
    s, i, r = (float(v) for v in (s, i, r))
    n = s + i + r
    d = i_day
    

    s_a = np.empty(n_days, "float")
    i_a = np.empty(n_days, "float")
    r_a = np.empty(n_days, "float")
    beta_a = np.empty(n_days, "float")

    index = 0
    for _ in range(n_days):
        beta = 
        d_a[index] = d
        s_a[index] = s
        i_a[index] = i
        r_a[index] = r
        index += 1

        s, i, r = sir(s, i, r, beta, gamma, n)
        d += 1

    d_a[index] = d
    s_a[index] = s
    i_a[index] = i
    r_a[index] = r
    return {
        "day": d_a,
        "susceptible": s_a,
        "infected": i_a,
        "recovered": r_a,
        "ever_infected": i_a + r_a
    }


def build_sim_sir_w_date_df(
    raw_df: pd.DataFrame,
    current_date: datetime,
    keys: Sequence[str],
) -> pd.DataFrame:
    day = raw_df.day
    return pd.DataFrame({
        "day": day,
        "date": day.astype('timedelta64[D]') + np.datetime64(current_date),
        **{
            key: raw_df[key]
            for key in keys
        }
    })

sim_sir()